In [41]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
import win32com.client as cli

outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
msg = inbox.Items #메세지 정보
msg_counter = msg.count # 총갯수

In [8]:
shape_name = 'SVC_CCH_23011901'
idx = int(shape_name.split('_')[2])
idx

23011901

In [36]:
ws_main = wb_cy.sheets['통합제어']
idx_col = ws_main.range('J12:J500').options(numbers=int).value
ws_main.range('J12').column

10

In [13]:
row_nm = idx_col.count(idx)

In [34]:
chs_rng = ws_main.range(11 +row_nm,17)
current_status = chs_rng.value
current_status

'REQUESTED'

In [27]:
ML_STATUS= ['REQUESTED','PROCESSING', 'SHIPPED']

ML_FOLDERS = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']

In [39]:
ml_dict =dict(zip(ML_STATUS,ML_FOLDERS[1:]))
ml_dict[current_status]

'1_Requests'

In [49]:
argu_folder = inbox.Parent.Folders(ml_dict[current_status])

In [50]:
msg = argu_folder.Items
part_request = []
service_request = 'SVC'
for ms in msg:
    if service_request in ms.Subject:
        part_request.append(ms)
# 서비스만 추출하기



In [52]:
part_request[0].Subject

'SVC_CCH_23011901'